This notebook implements the classic _Steers and Trees_ problem, as described in Davis and Johnson (2001) and Dykstra (1984) textbooks. 

In the first part of this lab, we solve this problem graphically using an interactive widget.

In the second part of the lab, we use the open source PuLP package to formulate and solve both LP and MIP formulations.

# The Steers and Trees Problem

The following description is adapted from Davis and Johnson (2001).

Sylvia Forester, a part-time farmer and forester from around Merritt, has 24 hectares of fallow land available and wants to use it to increase her income. She can either plant hybrid Christmas tree seedlings that mature in 1 year, or fatten steers by reserving part of her land for pasture. The trees are planted and sold in lots of 1000. It takes 1.5 hectares to grow a lot of trees and 4.0 hectares to fatten a steer. Sylvia is busy and only has 200 hours per year to spend on this enterprise. Experience shows that is takes 20 hours to cultivate, prune, harvest, and package one lot of trees and also 20 hours of effort per steer. She has \\$1200 of operating budget available for the year, and annual expenses are \\$30 per lot of trees and \\$240 per steer. She has already signed a contract with her neighbour for two steers. At current prices, the trees will return a net revenue of \\$0.50 each and steers will return a net revenue of \\$1000 each. Sylvia wants to maximize net revenue from this operation at the end of the year.

The goal is to maximize net revenue from this operation.

# Procedure

Download and install the PuLP package.

In [ ]:
!pip install pulp

Import required packages.

In [ ]:
%matplotlib inline
from pulp import *
from ipywidgets import interact, widgets
import numpy as np
import matplotlib.pyplot as plt

## Part I: Solve the problem graphically

Define a helper function to solve a linear equation for `x2`.

In [ ]:
def y(x1, c1, c2, rhs):
    return (rhs - c1*x1)/c2

Sample the `x1` dimension.

In [ ]:
x = np.arange(0, 10, 1)

In [ ]:
def showfig(z=0):
    fig, ax = plt.subplots(figsize=(12, 8))
    plt.plot(x, y(x, 4, 1.5, 24), ':', label='land')
    plt.plot(x, y(x, 240, 30, 1200), ':', label='budget')
    plt.plot(x, y(x, 20, 20, 200), ':', label='labour')
    plt.plot(x, y(x, 1000, 500, z), '-', label='objective') # objective function

    plt.plot([2, 2], [0, 8], 'o-')
    plt.plot([2, 3.6], [8, 6.4], 'o-')
    plt.plot([3.6, 4.5], [6.4, 4], 'o-')
    plt.plot([4.5, 5], [4, 0], 'o-')
    plt.plot([2, 5], [0, 0], 'o-')

    plt.axvline(x=2, linestyle=':')
    plt.xlim(0, 6)
    plt.ylim(0, 12)
    plt.legend()

In [ ]:
interact(showfig, z=widgets.IntSlider(min=0, max=10000, step=100, value=0))

## Part II: Solve the problem with a solver

Define a new `LpProblem` instance.

In [ ]:
prob = LpProblem('Steers and Trees', LpMaximize)

Define `x1` and `x2` variables.

In [ ]:
x1 = LpVariable('x1', 0, None, cat='C')
x2 = LpVariable('x2', 0, None, cat='C')

Define constraints.

In [ ]:
prob += 1000*x1 + 500*x2,         'revenue ' 
prob +=    4*x1 + 1.5*x2 <=   24, 'land    '
prob +=  240*x1 +  30*x2 <= 1200, 'budget  ' 
prob +=   20*x1 +  20*x2 <=  200, 'labour  '
prob +=      x1          >=    2, 'contract'

Output the optimization problem to a file (for inspection later).

In [ ]:
prob.writeLP('steersandtrees.lp')

Solve the optimization problem (using the default `PuLP` algorithm).

In [ ]:
prob.solve()

Display the objective function value.

In [ ]:
value(prob.objective)

Display some additional information about the solution.

In [ ]:
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)

print("objective=", value(prob.objective))

print("\nSensitivity Analysis\nConstraint\t\t\t\tSPrice\tSlack")
for name, c in list(prob.constraints.items()):
    print(name, ":", str(c).ljust(20), "\t", str(c.pi).rjust(5), "\t", str(c.slack).rjust(5))